# Joining Catalog-Based Views

This notebook demonstrates how to join data from different Views based on `external_id` and `space`.

## Prerequisites

- Multiple Views must be registered in Unity Catalog (see `registration_and_views.ipynb`)


In [ ]:
# Join two Views on external_id
query = """
SELECT 
    v.external_id,
    v.name AS vessel_name,
    s.external_id AS sensor_id,
    s.value AS sensor_value
FROM main.sailboat_sailboat_1.vessel v
JOIN main.power_windturbine_1.sensor s 
    ON v.external_id = s.vessel_external_id
WHERE v.space = 'sailboat'
LIMIT 10;
"""

result = spark.sql(query)
result.show(truncate=False)


In [ ]:
# Join Views with time series UDTFs using CROSS JOIN LATERAL
query = """
SELECT 
    v.external_id AS vessel_id,
    v.name AS vessel_name,
    ts.timestamp,
    ts.value AS speed
FROM main.sailboat_sailboat_1.vessel v
CROSS JOIN LATERAL (
    SELECT * FROM main.cdf_models.time_series_datapoints_udtf(
        space => v.speed_ts_space,
        external_id => v.speed_ts_external_id,
        start => '1d-ago',
        end => 'now',
        client_id => SECRET('cdf_sailboat_sailboat', 'client_id'),
        client_secret => SECRET('cdf_sailboat_sailboat', 'client_secret'),
        tenant_id => SECRET('cdf_sailboat_sailboat', 'tenant_id'),
        cdf_cluster => SECRET('cdf_sailboat_sailboat', 'cdf_cluster'),
        project => SECRET('cdf_sailboat_sailboat', 'project')
    )
) ts
WHERE v.space = 'sailboat'
ORDER BY v.external_id, ts.timestamp
LIMIT 100;
"""

result = spark.sql(query)
result.show(truncate=False)
